# S3 에 저장된 모델을 SageMaker INF2 에 배포하기

이 노트북은 [MLP-KTLim/llama-3-Korean-Bllossom-8B](https://huggingface.co/MLP-KTLim/llama-3-Korean-Bllossom-8B) 의 한국어 파인 튜닝 모델을 AWS Neuron Model 로 바꾼 [Gonsoo/AWS-Neuron-llama-3-Korean-Bllossom-8B](https://huggingface.co/Gonsoo/AWS-Neuron-llama-3-Korean-Bllossom-8B) 모델을 사용 합니다.<br>
Amazon EC2 Inferentia2 에 Hugging Face Text Generation Inference (HF TGI) 도커 컨테이너를 이용하여, SageMaker Endpoint 에 배포 합니다.

#### 코드 참조
- [Deploy Llama 3 70B on AWS Inferentia2 with Hugging Face Optimum](https://github.com/philschmid/huggingface-inferentia2-samples/blob/main/llama3-70b/deploy-llama-3-70b-inferentia2.ipynb)



## 1. 다음과 같은 패키지를 설치 합니다.
- 처음 실행시에 아래의 주석을 제거하고 실행 하세요



In [33]:
# ! pip install huggingface_hub
# !pip install "sagemaker>=2.199.0" transformers --upgrade --quiet

## 2. SageMaker 세션 등 가져오기

In [34]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
region = sess.boto_region_name
print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {region}")


[05/22/25 01:39:28] INFO     Found credentials from IAM Role:                                   ]8;id=138610;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=172651;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[05/22/25 01:39:29] INFO     Found credentials from IAM Role:                                   ]8;id=790876;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=376499;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=780856;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=994096;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker role arn: arn:aws:iam::057716757052:role/workshop-sagemaker-kfp-role2
sagemaker session region: us-west-2


## 3. Hugging Face LLM Inf2 DLC (Deep Learning Container ) 가져오기

새로운 Hugging Face TGI Neuronx DLC를 사용하여 AWS Inferentia2에서 추론을 실행할 수 있습니다. sagemaker SDK의 get_huggingface_llm_image_uri 메서드를 사용하여 원하는 backend, session, region, 그리고 version에 따라 적절한 Hugging Face TGI Neuronx DLC URI를 검색할 수 있습니다. 사용 가능한 모든 버전은 여기에서 확인할 수 있습니다.
- [중요] 아래에서는 버전 version="0.0.24" 를 사용합니다. 이유는 모델의 뉴런 컴파일 버전과 일치를 시키기 위함 입니다.
    - 0.0.24 는 Neuron-cc 2.14 버전이 탑재되어 있습니다.
- 참조: [Hugging Face TGI Neuronx DLC URI](https://github.com/aws/deep-learning-containers/releases?q=tgi+AND+neuronx&expanded=true)



In [49]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface-neuronx",
  version="0.0.23"
  # version="0.0.24"
)
print(f"llm image uri: {llm_image}")

# 신규 버전 ( 2024년 8월 22일 출시) : Neuron-CC 2.14 버전
# 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:\
# 2.1.2-optimum0.0.24-neuronx-py310-ubuntu22.04-v1.0
llm_image = f"763104351884.dkr.ecr.{region}.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.2-optimum0.0.24-neuronx-py310-ubuntu22.04-v1.0"
print(f"llm image uri: {llm_image}")

[05/22/25 04:01:50] INFO     Found credentials from IAM Role:                                   ]8;id=898228;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=931107;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Defaulting to only available Python version: py310                   ]8;id=174687;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=444967;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/image_uris.py#610\610]8;;\

                    INFO     Defaulting to only supported image scope: inf2.                      ]8;id=660640;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=142725;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/image_uris.py#534\534]8;;\

llm image uri: 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.2-optimum0.0.23-neuronx-py310-ubuntu22.04
llm image uri: 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.2-optimum0.0.24-neuronx-py310-ubuntu22.04-v1.0


## 4. HF 에서 모델 다운로드 및 S3 업로드

### 모델 다운로드
- NeuronCC-2-14 버전으로 컴파일된 모델을 다운로드 합니다.

In [50]:
import os

download_folder = "/home/ec2-user/SageMaker/models/AWS-NeuronCC-2-14-llama-3-Korean-Bllossom-8B"

# model_name = "Gonsoo/AWS-Neuron-llama-3-Korean-Bllossom-8B" # Neuron-CC 2.13 버전 에서 컴파일
# 아래의 도커의 optimum-cli 를 통하여 컴파일 됨
# 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.2-optimum0.0.24-neuronx-py310-ubuntu22.04-v1.0
model_name = "Gonsoo/AWS-NeuronCC-2-14-llama-3-Korean-Bllossom-8B" # Neuron-CC 2.14 버전 에서 컴파일

os.makedirs(download_folder, exist_ok=True)


In [51]:
! huggingface-cli download {model_name} --local-dir {download_folder} --exclude ".cache" --exclude ".gitattributes"
! ls -al {download_folder} 

Fetching 8 files: 100%|█████████████████████████| 8/8 [00:00<00:00, 4911.36it/s]
/home/ec2-user/SageMaker/models/AWS-NeuronCC-2-14-llama-3-Korean-Bllossom-8B
total 8960
drwxrwxr-x 4 ec2-user ec2-user    4096 May 21 14:30 .
drwxrwxr-x 3 ec2-user ec2-user    4096 May 21 14:30 ..
drwxrwxr-x 3 ec2-user ec2-user    4096 May 21 14:30 .cache
drwxrwxr-x 2 ec2-user ec2-user    4096 May 21 14:30 compiled
-rw-rw-r-- 1 ec2-user ec2-user    1093 May 21 14:30 config.json
-rw-rw-r-- 1 ec2-user ec2-user     172 May 21 14:30 generation_config.json
-rw-rw-r-- 1 ec2-user ec2-user    1864 May 21 14:30 README.md
-rw-rw-r-- 1 ec2-user ec2-user     444 May 21 14:30 special_tokens_map.json
-rw-rw-r-- 1 ec2-user ec2-user   51106 May 21 14:30 tokenizer_config.json
-rw-rw-r-- 1 ec2-user ec2-user 9085699 May 21 14:30 tokenizer.json


### 모델을 S3 에 업로드

In [39]:
desired_s3_uri = f"s3://{sagemaker_session_bucket}/inf2_model"

In [40]:
from sagemaker.s3 import S3Uploader

file_s3_path = S3Uploader.upload(local_path=download_folder, desired_s3_uri=desired_s3_uri)
print("file_s3_path: ", file_s3_path)


[05/22/25 01:40:56] INFO     Found credentials from IAM Role:                                   ]8;id=319112;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=652875;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

file_s3_path:  s3://sagemaker-us-west-2-057716757052/inf2_model


In [41]:
# ! aws s3 rm {file_s3_path} --recursive
! aws s3 ls {file_s3_path} --recursive

2025-05-22 01:40:58          1 inf2_model/.cache/huggingface/.gitignore
2025-05-22 01:40:58          0 inf2_model/.cache/huggingface/download/README.md.lock
2025-05-22 01:40:58        100 inf2_model/.cache/huggingface/download/README.md.metadata
2025-05-22 01:40:58          0 inf2_model/.cache/huggingface/download/compiled/2ae6fb8fd3c66e17e30f.neff.lock
2025-05-22 01:40:58        125 inf2_model/.cache/huggingface/download/compiled/2ae6fb8fd3c66e17e30f.neff.metadata
2025-05-22 01:40:58          0 inf2_model/.cache/huggingface/download/compiled/675e2498ec72a928a923.neff.lock
2025-05-22 01:40:59        125 inf2_model/.cache/huggingface/download/compiled/675e2498ec72a928a923.neff.metadata
2025-05-22 01:40:58          0 inf2_model/.cache/huggingface/download/config.json.lock
2025-05-22 01:40:58        100 inf2_model/.cache/huggingface/download/config.json.metadata
2025-05-22 01:40:58          0 inf2_model/.cache/huggingface/download/generation_config.json.lock
2025-05-22 01:40:58        101

## 5. 한국어 모델을 inferentia2 에 배포하기


모델을 Amazon SageMaker에 배포하기 전에 TGI Neuronx 엔드포인트 구성을 정의해야 합니다. 다음과 같은 추가 매개변수를 정의해야 합니다:

- HF_NUM_CORES: 컴파일에 사용된 Neuron 코어의 수.
- HF_BATCH_SIZE: 모델 컴파일에 사용된 배치 크기.
- HF_SEQUENCE_LENGTH: 모델 컴파일에 사용된 시퀀스 길이.
- HF_AUTO_CAST_TYPE: 모델 컴파일에 사용된 자동 캐스트 유형.

또한 기존의 TGI 매개변수도 정의해야 합니다:

- HF_MODEL_ID: "/opt/ml/model"
    - ECR 에서 추론 이미지를 다운로드 받고, 컨테이너가 생성이 됩니다. 이후에 S3 의 모델 파일은 "/opt/ml/model" 로 다운로드가 됩니다. 그래서 이 경로를 기술해야만 합니다.
- MAX_BATCH_SIZE: 모델이 처리할 수 있는 최대 배치 크기로, 컴파일에 사용된 배치 크기와 동일합니다.
- MAX_INPUT_LENGTH: 모델이 처리할 수 있는 최대 입력 길이.
- MAX_TOTAL_TOKENS: 모델이 생성할 수 있는 최대 총 토큰 수로, 컴파일에 사용된 시퀀스 길이와 동일합니다.

### 모델 배포 파라미터 설정 및 SageMaker HuggingFaceModel 생성

In [42]:
from huggingface_hub import HfFolder
from sagemaker.huggingface import HuggingFaceModel

# sagemaker config
instance_type = "ml.inf2.xlarge"
health_check_timeout=600 # additional time to load the model
volume_size=64 # size in GB of the EBS volume

# Define Model and Endpoint configuration parameter
config = {
    "HF_MODEL_ID": "/opt/ml/model",       # Path to the model in the container
    "HF_NUM_CORES": "2", # number of neuron cores
    "HF_BATCH_SIZE": "4", # batch size used to compile the model
    "HF_SEQUENCE_LENGTH": "4096", # length used to compile the model
    "HF_AUTO_CAST_TYPE": "fp16",  # dtype of the model
    "MAX_BATCH_SIZE": "4", # max batch size for the model
    "MAX_INPUT_LENGTH": "4000", # max length of input text
    "MAX_TOTAL_TOKENS": "4096", # max length of generated text
    "MESSAGES_API_ENABLED": "true", # Enable the messages API
    # 검증된 오프라인 모드 설정
    "HF_HUB_OFFLINE": "1",               # Hub 접근 완전 차단
    "TRANSFORMERS_OFFLINE": "1",
    # 추가 Optimum 관련 설정
    "OPTIMUM_OFFLINE": "1",                    # Optimum 오프라인 모드
    "HF_HUB_DISABLE_PROGRESS_BARS": "1",       # 진행률 표시 비활성화
    "HF_HUB_DISABLE_SYMLINKS_WARNING": "1",    # 심볼릭 링크 경고 비활성화
    "TOKENIZERS_PARALLELISM": "false",     
}

# model_data 딕셔너리 생성
model_data = {
    'S3DataSource': {
        'S3Uri': f"{file_s3_path}/",
        'S3DataType': 'S3Prefix',
        'CompressionType': 'None'
    }
}

print("model_data: \n", model_data) 
# create HuggingFaceModel with the image uri

llm_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  model_data=model_data,
  env=config
)

model_data: 
 {'S3DataSource': {'S3Uri': 's3://sagemaker-us-west-2-057716757052/inf2_model/', 'S3DataType': 'S3Prefix', 'CompressionType': 'None'}}


[05/22/25 01:41:03] INFO     Found credentials from IAM Role:                                   ]8;id=277871;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=638642;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

### 모델을 SageMaker Endpoint 에 배포

HuggingFaceModel을 생성한 후에는 deploy 메서드를 사용하여 Amazon SageMaker에 배포할 수 있습니다. 우리는 ml.inf2.xlarge 인스턴스 유형을 사용하여 모델을 배포할 것입니다. TGI는 자동으로 모든 Inferentia 디바이스에 걸쳐 모델을 분산하고 샤딩할 것입니다.
- 다음은 약 17분이 소요 됩니다.

In [43]:
# deactivate warning since model is compiled
llm_model._is_compiled_model = True

llm = llm_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout,
  volume_size=volume_size
)

[05/22/25 01:41:15] INFO     Creating model with name:                                              ]8;id=53187;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=42547;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py#4094\4094]8;;\
                             huggingface-pytorch-tgi-inference-ml-in-2025-05-22-01-41-15-314                       

[05/22/25 01:41:16] INFO     Creating endpoint-config with name                                     ]8;id=442362;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=479933;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py#6019\6019]8;;\
                             huggingface-pytorch-tgi-inference-ml-in-2025-05-22-01-41-16-176                       

                    INFO     Creating endpoint with name                                            ]8;id=777526;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=656184;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py#4841\4841]8;;\
                             huggingface-pytorch-tgi-inference-ml-in-2025-05-22-01-41-16-176                       

-----------------------------------*

[05/22/25 01:59:19] ERROR    Please check the troubleshooting guide for common errors:              ]8;id=326959;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=85410;file:///home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.py#4864\4864]8;;\
                             https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-t                
                             roubleshooting.html#sagemaker-python-sdk-troubleshooting-create-endpoi                
                             nt                                                                                    

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│    1 # deactivate warning since model is compiled                                                │
│    2 llm_model._is_compiled_model = True                                                         │
│    3                                                                                             │
│ ❱  4 llm = llm_model.deploy(                                                                     │
│    5   initial_instance_count=1,                                                                 │
│    6   instance_type=instance_type,                                                              │
│    7   container_startup_health_check_timeout=health_check_timeout,                              │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/huggingfa │
│ ce/model.py:326 in deploy                                                                        │
│                                                                                                  │
│   323 │   │   │   │   inference_tool=inference_tool,                                             │
│   324 │   │   │   )                                                                              │
│   325 │   │                                                                                      │
│ ❱ 326 │   │   return super(HuggingFaceModel, self).deploy(                                       │
│   327 │   │   │   initial_instance_count,                                                        │
│   328 │   │   │   instance_type,                                                                 │
│   329 │   │   │   serializer,                                                                    │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/model.py: │
│ 1814 in deploy                                                                                   │
│                                                                                                  │
│   1811 │   │   │   │   )                                                                         │
│   1812 │   │   │   │   self.sagemaker_session.update_endpoint(self.endpoint_name, endpoint_conf  │
│   1813 │   │   │   else:                                                                         │
│ ❱ 1814 │   │   │   │   self.sagemaker_session.endpoint_from_production_variants(                 │
│   1815 │   │   │   │   │   name=self.endpoint_name,                                              │
│   1816 │   │   │   │   │   production_variants=[production_variant],                             │
│   1817 │   │   │   │   │   tags=tags,                                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/neuron_pytorch_p38/lib/python3.8/site-packages/sagemaker/session.p │
│ y:6022 in endpoint_from_production_variants                                                      │
│                                                                                                  │
│   6019 │   │   logger.info("Creating endpoint-config with name %s", name)                        │
│   6020 │   │   self.sagemaker_client.create_endpoint_config(**config_options)                    │
│   6021 │   │                                                                                     │
│ ❱ 6022 │   │   return self.create_endpoint(                

## 6. 추론

엔드포인트가 배포된 후에는 이를 통해 추론을 실행할 수 있습니다. 우리는 predictor의 predict 메서드를 사용하여 엔드포인트에서 추론을 실행할 것입니다. 생성에 영향을 미치는 다양한 매개변수로 추론을 수행할 수 있습니다. 매개변수는 페이로드의 parameters 속성에서 정의할 수 있습니다. 지원되는 매개변수는 여기에서 확인할 수 있습니다.
메시지 API를 사용하면 대화 방식으로 모델과 상호작용할 수 있습니다. 메시지의 역할과 내용을 정의할 수 있습니다. 역할은 system, assistant 또는 user일 수 있습니다. system 역할은 모델에 컨텍스트를 제공하는 데 사용되고, user 역할은 질문을 하거나 모델에 입력을 제공하는 데 사용됩니다.

```json
{
  "messages": [
    { "role": "system", "content": "You are a helpful assistant." },
    { "role": "user", "content": "What is deep learning?" }
  ]
}
```

In [ ]:
# Prompt to generate
messages=[
    { "role": "system", "content": "당신은 인공지능 전문가 입니다." },
    { "role": "user", "content": "딥러닝이 무엇인지 말해 주세요?" }
  ]

# Generation arguments
parameters = {
    "model": "Gonsoo/AWS-Neuron-llama-3-Korean-Bllossom-8B", # placholder, needed
    "top_p": 0.6,
    "temperature": 0.9,
    "max_tokens": 2048,
    "stop": ["<|eot_id|>"],
}

이제 테스트 해보시죠

In [ ]:
%%time
chat = llm.predict({"messages" :messages, **parameters,"steam":True})

print(chat["choices"][0]["message"]["content"].strip())

## 6. 리소스 정리

SageMaker HuggingFace Model 및 엔드포인트를 삭제 합니다.


In [18]:
llm.delete_model()
llm.delete_endpoint()